### Removing duplicate triples when is obtained treatments by parallel codes

In [1]:
import pandas as pd
from sklearn.model_selection import KFold
from rdflib import Graph

from collections import OrderedDict

In [2]:
#path = 'dataset/small_dataset/Study_2/'
path = ''

In [3]:
def save_ttl_file(graph_ttl, path):
    #open text file
    ttl_file = open(path + "G3_00.ttl", "w")
    #write string to file
    n = ttl_file.write(graph_ttl)
    #close file
    ttl_file.close()

In [4]:
graph_ttl = open(path + "G3.ttl").read()
graph_ttl = "\n".join(list(OrderedDict.fromkeys(graph_ttl.split("\n"))))
save_ttl_file(graph_ttl, path)

# Cross-Validation

In [5]:
g = Graph()
g.parse("G3.ttl", format="ttl")

<Graph identifier=N93e6b20273ff4577b26f31befea7984d (<class 'rdflib.graph.Graph'>)>

In [6]:
from rdflib.plugins.sparql.processor import SPARQLResult

def sparql_results_to_df(results: SPARQLResult) -> pd.DataFrame:
    """
    Export results from an rdflib SPARQL query into a `pandas.DataFrame`,
    using Python types. See https://github.com/RDFLib/rdflib/issues/1179.
    """
    return pd.DataFrame(
        data=([None if x is None else x.toPython() for x in row] for row in results),
        columns=[str(x) for x in results.vars],
    )

In [7]:
query = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example/#> 
PREFIX treatment_drug: <http://example/Treatment_Drug#>
    
select distinct ?treatment
where {
    ?treatment rdf:type ex:Treatment .
    ?treatment ex:hasClassificationEffect ex:effective .
    }
    """

qres = g.query(query)
#df_toxicity = pd.DataFrame(qres.bindings)
df_effective = sparql_results_to_df(qres)
df_effective['predicate'] = 'ex:hasClassificationEffect'
df_effective['object'] = 'ex:effective .'
#df_toxicity.replace('http://example/Treatment_Drug#', 'treatment_drug:', regex=True, inplace=True)

In [10]:
df_decrease_effectiveness

,treatment,predicate,object
0,http://example/Treatment/treatment0,ex:hasClassificationEffect,ex:decrease_effectiveness .
1,http://example/Treatment/treatment1,ex:hasClassificationEffect,ex:decrease_effectiveness .
2,http://example/Treatment/treatment2,ex:hasClassificationEffect,ex:decrease_effectiveness .
3,http://example/Treatment/treatment3,ex:hasClassificationEffect,ex:decrease_effectiveness .
4,http://example/Treatment/treatment4,ex:hasClassificationEffect,ex:decrease_effectiveness .
...,...,...,...
519,http://example/Treatment/treatment519,ex:hasClassificationEffect,ex:decrease_effectiveness .
520,http://example/Treatment/treatment520,ex:hasClassificationEffect,ex:decrease_effectiveness .
521,http://example/Treatment/treatment521,ex:hasClassificationEffect,ex:decrease_effectiveness .
522,http://example/Treatment/treatment522,ex:hasClassificationEffect,ex:decrease_effectiveness .


In [8]:
query = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example/#> 
PREFIX treatment_drug: <http://example/Treatment_Drug#>
    
select distinct ?treatment
where {
    ?treatment rdf:type ex:Treatment .
    ?treatment ex:hasClassificationEffect ex:decrease_effectiveness .
    }
    """
#<http://example/Treatment/treatment301>

qres = g.query(query)

#df_effectiveness = pd.DataFrame(qres.bindings)
df_decrease_effectiveness = sparql_results_to_df(qres)
df_decrease_effectiveness['predicate'] = 'ex:hasClassificationEffect'
df_decrease_effectiveness['object'] = 'ex:decrease_effectiveness .'
#df_effectiveness.replace('http://example/Treatment_Drug#', 'treatment_drug:', regex=True, inplace=True)

In [9]:
df_category = pd.concat([df_effective, df_decrease_effectiveness])
df_category['treatment'] = '<' + df_category['treatment'].astype(str) + '>'
display(df_category.head(2), df_category.shape)

,treatment,predicate,object
0,<http://example/Treatment/treatment524>,ex:hasClassificationEffect,ex:effective .
1,<http://example/Treatment/treatment525>,ex:hasClassificationEffect,ex:effective .


(580, 3)

In [10]:
df_category.reset_index(inplace=True)
df_category.drop(columns=['index'], inplace=True)
display(df_category.head(2), df_category.shape)

,treatment,predicate,object
0,<http://example/Treatment/treatment524>,ex:hasClassificationEffect,ex:effective .
1,<http://example/Treatment/treatment525>,ex:hasClassificationEffect,ex:effective .


(580, 3)

In [11]:
def cross_validation(dataset):
    test_set = []
    training_set = []
    X = dataset
    kf = KFold(n_splits=5, shuffle=True, random_state=0)
    kf.get_n_splits(X)
    
    split = 1
    for train_index, test_index in kf.split(X):
        #print("TRAIN:", train_index, "TEST:", test_index)
        #print(len(test_index))
        #test_set.append(X.iloc[test_index])
        #training_set.append(X.iloc[train_index])
        test = X.iloc[test_index]
        test['object'] = test['object'].str.replace(' .', '')
        test.to_csv(path + 'G3/test_'+str(split)+'.ttl', sep='\t', index=False, header=False)
        create_training_set(split)
        split+=1

In [12]:
def create_training_set(split):
    with open(path + 'G3/test_'+str(split)+'.ttl', "r") as f:
        lines_test = f.readlines()
    with open(path + "G3.ttl", "r") as f:
        lines_data = f.readlines()
    with open(path + 'G3/train_'+str(split)+'.ttl', "w") as f:
        for line in lines_data:
            line = line.replace(' .', '')
            if line in  lines_test or '@prefix' in line:
                continue
            else:                
                f.write(line)

In [13]:
cross_validation(df_category)

C:\Users\rivasa\AppData\Local\Temp/ipykernel_14844/134286601.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  test['object'] = test['object'].str.replace(' .', '')
C:\Users\rivasa\AppData\Local\Temp/ipykernel_14844/134286601.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['object'] = test['object'].str.replace(' .', '')


In [63]:
g.serialize(destination="dataset/KG_treatment_serialize.ttl", format="ttl")

In [31]:
from platform import python_version

print(python_version())

3.9.7


In [8]:
from sklearn.model_selection import train_test_split

data_g1 = pd.read_csv('G1_data.ttl', delimiter="\t", header=None)
data_g1.columns=['s','p','o']
data_g1['o'] = data_g1['o'].str.replace(' .', '')
data_g1

C:\Users\rivasa\AppData\Local\Temp/ipykernel_23212/2858445035.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  data_g1['o'] = data_g1['o'].str.replace(' .', '')


,s,p,o
0,ex:higher_toxicity,rdf:type,ex:HigherToxicity
1,ex:lower_effectiveness,rdf:type,ex:LowerEffectiveness
2,<http://example/Treatment/treatment0>,rdf:type,ex:Treatment
3,<http://example/Treatment/treatment0>,ex:hasClassificationEffect,ex:decrease_effectiveness
4,<http://example/Drug/DB00331>,rdf:type,ex:Drug
...,...,...,...
15112,treatment_drug:treatment579_DB00530,ex:hasLowerEffect,ex:lower_effectiveness
15113,ddi:treatment579DB00338DB00530,rdf:type,ex:DDI
15114,<http://example/Treatment/treatment579>,ex:related_to,ddi:treatment579DB00338DB00530
15115,ddi:treatment579DB00338DB00530,ex:precipitant_drug,<http://example/Drug/DB00338>


# Split set

In [4]:
train, test = train_test_split(data_g1, test_size=0.2)
test.to_csv('G2/test_1.ttl', sep='\t', index=False, header=False)
train.to_csv('G2/train_1.ttl', sep='\t', index=False, header=False)
display(train, test)

,s,p,o
15987,ddi:treatment535DB00958DB01050,ex:precipitant_drug,<http://example/Drug/DB00958>
15073,treatment_drug:treatment474_DB06186,rdf:type,ex:Treatment_Drug
15778,treatment_drug:treatment527_DB00584,ex:related_to,<http://example/Treatment/treatment527>
10274,treatment_drug:treatment216_DB00338,ex:related_to,<http://example/Drug/DB00338>
12829,treatment_drug:treatment327_DB00635,ex:related_to,<http://example/Drug/DB00635>
...,...,...,...
13369,ddi:treatment360DB00641DB01229,rdf:type,ex:DDI
4851,ddi:treatment58DB00338DB01229,ex:precipitant_drug,<http://example/Drug/DB00338>
5521,ddi:treatment71DB00335DB01076,ex:precipitant_drug,<http://example/Drug/DB00335>
10544,treatment_drug:treatment226_DB01248,ex:related_to,<http://example/Drug/DB01248>


,s,p,o
6859,treatment_drug:treatment105_DB00381,rdf:type,ex:Treatment_Drug
13212,treatment_drug:treatment351_DB01229,ex:hasLowerEffect,ex:lower_effectiveness
10880,ddi:treatment237DB00338DB00361,ex:precipitant_drug,<http://example/Drug/DB00338>
7828,ddi:treatment130DB00695DB00642,ex:object_drug,<http://example/Drug/DB00642>
15141,treatment_drug:treatment480_DB01229,ex:related_to,<http://example/Treatment/treatment480>
...,...,...,...
2627,ddi:treatment22DB01229DB00381,rdf:type,ex:DDI
1555,<http://example/Treatment/treatment9>,ex:related_to,ddi:treatment9DB00316DB01076
2278,treatment_drug:treatment18_DB00773,ex:related_to,<http://example/Treatment/treatment18>
4456,ddi:treatment52DB00361DB01233,ex:precipitant_drug,<http://example/Drug/DB00361>


# Cross-validation

In [9]:
def cross_validation(dataset):
    X = dataset
    kf = KFold(n_splits=5, shuffle=True, random_state=0)
    kf.get_n_splits(X)
    split = 1
    for train_index, test_index in kf.split(X):
        test = X.iloc[test_index]
        train = X.iloc[train_index]
        test.to_csv('G1/test_'+str(split)+'.ttl', sep='\t', index=False, header=False)
        train.to_csv('G1/train_'+str(split)+'.ttl', sep='\t', index=False, header=False)
        split+=1

cross_validation(data_g1)